### Install enoslib (+ small setup)

In [ ]:
#!pip install -U pip
#!pip install enoslib
#!pip install python-grid5000

In [ ]:
# need for credentials in .env
#!echo -e "username: $(head -n 1 .env)\npassword: $(head -n 2 .env | tail -n 1)" > ~/.python-grid5000.yaml

In [ ]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

### Launch machines

In [ ]:
job_name = 'dev'

conf = (
    en.G5kConf.from_settings(
        job_name=job_name,
        job_type=["deploy"],
        env_name="ubuntu2204-min",
        walltime="2:00:00",
    )    
    .add_machine(roles=["client"], cluster="gros", nodes=1)
    .add_machine(roles=["server"], cluster="gros", nodes=1)
)

provider = en.G5k(conf)

roles, networks = provider.init()

### mount local files on grid5000 

In [23]:
import os

site = conf.sites[0]
for role in roles:
  hosts = [_.alias.split('.')[0] for _ in roles[role]]
  for host in hosts:
    os.system(f'bash mount_sshfs.sh {site} {host} {role}')

gros-87
gros-90


### Setup the machines

In [ ]:
with en.actions(roles=roles) as p:    
    p.shell("apt update")

### Kill every last one of them

In [24]:
# Release all Grid'5000 resources
provider.destroy()